Version control :
v0.0.1
v0.0.2
v0.0.3
v0.0.4

In [2]:
import os 
import zipfile as zf
from cryptography.fernet import Fernet as fn
import random
import string
import json
import time

In [3]:
#global variables
ver_control = 0
software_name = 0
creator = 0
def globalVariable(ver="v0.0.4", soft_name="Project Ultimate Backup", author="Anjal.P"):
    global ver_control, software_name, creator 
    ver_control = ver
    software_name = soft_name
    creator = author
    return ver_control, software_name, creator

In [ ]:
def errorSave(where_to_save, error_report, root_directory):
    global error
    where_to_save = where_to_save + "//"
    print("look at: ", where_to_save + "Error Report UB.txt")
    ver_control, software_name, creator, section = versionControl()
    date_error = str(time.localtime(time.time()).tm_mday) + "." + str(time.localtime(time.time()).tm_mon) + "." + str(time.localtime(time.time()).tm_year)
    if os.path.exists(where_to_save + "Error Report UB.txt")==False:
        try:
            with open(where_to_save + "Error Report UB.txt", 'a') as error_file:
                error_file.write("--"*50 + "\n")
                error_file.write("Errors :" + "[" +str(error) + "]" + "\n")
                error_file.write("On Running Restore: " + root_directory + "\n")
                error_file.write("Created On: " + date_error + "\n")
                error_file.write("Report: \n")
                for eachError in error_report:
                    error_file.write("     > " + eachError + " " + error_report[eachError])
                error_file.write("This is an automated report generated by " + str(software_name) + " " + str(ver_control) + " " + "Author: " + str(creator) + "\n")
                error_file.write("--"*50 + "\n")
                error_file.close()
        except Exception as e:
            print("Error even creating the error log .txt at: " + where_to_save + "Error Report UB.txt")
            print("Error Report: " + str(error_report))
            print("Error on errorSave: " + str(e))
    else:
        try:
            with open(where_to_save + "Error Report UB.txt", 'a') as error_file:
                error_file.write("--"*50 + "\n")
                error_file.write("Errors :" + "[" +str(error) + "]" + "\n")
                error_file.write("On Running Backup: " + root_directory + "\n")
                error_file.write("Created On: " + date_error + "\n")
                error_file.write("Report: \n")
                for eachError in error_report:
                    error_file.write("     > " + eachError + " " + error_report[eachError])
                error_file.write("This is an automated report generated by " + str(software_name) + " " + str(ver_control) + " " + "Author: " + str(creator) + "\n")
                error_file.write("--"*50 + "\n")
                error_file.close()
        except:
            print("Error even creating the error log .txt at: " + where_to_save + "Error Report UB.txt")
            print("Error Report: " + str(error_report))
            print("Error on errorSave: " + str(e))

In [4]:
# v0.0.3 : Review : this update has the change scheme of the namng files i.e. now each file is zipped seperately as it 
#seems that the modification process and the reading process is thus efficient and hence so done. 
def directoryScan(root_directory):               #Scan the given directory ad reutrns the following variables 
    root_folders = {}                  #variable
    folder_no = 0
    root_files = {}
    file_no = 0
    root_size = 0 # in bytes.  
    files_properties = {}
    if os.path.isdir(root_directory)==False:     #define error in passes root_directory variable
        print("No such Directory exist.....")
    else:
        os.chdir(root_directory)
        for each_obj in os.listdir(root_directory):
            if os.path.isfile(each_obj)==True:    #if it is a file then do the below code
                root_files[file_no] = each_obj
                files_properties[file_no] = list(os.stat(each_obj))[6:]   #only contain 4 values size
                file_no = file_no + 1
            elif os.path.isdir(each_obj)==True:
                root_folders[folder_no] = each_obj                  #or if it is a folder :::
                folder_no += 1
    return root_folders, root_files, files_properties

#this functin return the :
#root_folders = folder contained in the given directory
#root_file = files contained in the root foder
#root_size = sizes of the all files in the root folder
#rar_name = name for the zip file containing the root_files
#files_properties = gives the propertieso of the file in the order: size, atime, mtime, ctime(a:accesed, m:modified, c:created)

In [5]:
def zipEncryptFiles(fil_dict, dir_loc, nu_ub_files, where_to_B, working_dir):    #Convert the given folder file to a zip and then encrypt the same
    limit_zip = 300*1024*1024   #In bytes
    size_to = 200*1024*1024   #In bytes
    symbol_list = "~!`@#$%^&()_-+={[}];',"
    zip_name = {}
    key = {}                                  #the stuff where zip file get stored
    error = 0
    error_report = {}
    if len(fil_dict)!=0:                                         #creating the files to zip 
        for files in fil_dict:
            os.chdir(dir_loc)                                     #changing the directory to the fodler in deal.
            error_report[files] = {}
            rand_range = random.randrange(8, 24)
            rand_name = ''.join(random.choices(string.ascii_letters + string.digits + symbol_list, k=rand_range))
            fi_name = rand_name + ".zip"
            try:
                rar_file = zf.ZipFile(working_dir + fi_name, 'w')
                try:
                    rar_file.write(fil_dict[files], compress_type=zf.ZIP_DEFLATED)
                except:
                    error_report[files]["er_write_zip"] = fil_dict[files]
                    error = 1 
                    rar_file.close()
                    errorSave(working_dir, error_report, dir_loc)
                    return zip_name, key, nu_ub_files
                rar_file.close()
            except:
                error_report[files]["er_create_zip"] = working_dir + fi_name
                error = 1                                                           #encryption key
                errorSave(working_dir, error_report, dir_loc)
                return zip_name, key, nu_ub_files
            key[files] = (fn.generate_key()).decode()   #as saving json do not read binary
            enc_key = fn(key[files].encode())
            os.chdir(working_dir)                           #encrypting the zip and writting to output directory(harddrive)
            if os.stat(fi_name).st_size>=limit_zip:                                    #if split needed
                zip_name[files] = {}
                num_split = os.stat(fi_name).st_size//size_to + 1 
                try: 
                    with open (fi_name, 'rb') as split:
                        for volume in range(1, num_split + 1, 1):
                            rand_range = random.randrange(8, 24)
                            rand_name = ''.join(random.choices(string.ascii_letters + string.digits + symbol_list, k=rand_range))
                            split_name = where_to_B + "//" + rand_name + "."
                            zip_name[files][volume] = rand_name 
                            encr_spl_file = enc_key.encrypt(split.read(size_to))
                            try:
                                with open (split_name + "sUB", 'wb') as writ_spl:
                                    try:
                                        writ_spl.write(encr_spl_file)
                                        writ_spl.close()
                                        nu_ub_files = nu_ub_files + 1
                                    except:
                                        error = 1
                                        error_report[files]["er_write_sUB"] = split_name + "sUB"
                                        writ_spl.close()
                                        split.close()
                                        errorSave(working_dir, error_report, dir_loc)
                                        return zip_name, key, nu_ub_files
                            except:
                                error = 1
                                error_report[files]["er_create_sUB"] = split_name + "sUB"
                                split.close()
                                errorSave(working_dir, error_report, dir_loc)
                                return zip_name, key, nu_ub_files
                    split.close()
                except:
                    error_report[files]["er_open_zip"] = working_dir + fi_name
                    error = 1
                    errorSave(working_dir, error_report, dir_loc)
                    return zip_name, key, nu_ub_files
                os.remove(fi_name)                                                      #delete the zip file 
            else:                                                                  #no split needed
                zip_name[files] = fi_name[:-4]
                try:
                    with open (fi_name, 'rb') as no_split:
                        no_split_name = where_to_B + "//" + fi_name[:-4] + "."
                        encr_spl_file = enc_key.encrypt(no_split.read())
                        try: 
                            with open (no_split_name + "eUB", 'wb') as writ_spl:
                                try:
                                    writ_spl.write(encr_spl_file)
                                    writ_spl.close()
                                    nu_ub_files = nu_ub_files + 1
                                except:
                                    error_report[files]["er_write_eUB"] = no_split_name + "eUB"
                                    error = 1
                                    writ_spl.close()
                                    no_split.close()
                                    errorSave(working_dir, error_report, dir_loc)
                                    return zip_name, key, nu_ub_files
                        except:
                            error_report[files]["er_create_eUB"] = no_split_name + "eUB"
                            error = 1
                            no_split.close()
                            errorSave(working_dir, error_report, dir_loc)
                            return zip_name, key, nu_ub_files
                    no_split.close()
                except:
                    error_report[files]["er_open_zip"] = working_dir + fi_name
                    error = 1
                    errorSave(working_dir, error_report, dir_loc)
                    return zip_name, key, nu_ub_files
                os.remove(fi_name)
    return zip_name, key, nu_ub_files

In [6]:
# v0.0.4 here the folder variable output directory with '\', and we have to change it to '//'.

def newBackup(root_directory, directory_to_backup, working_dir, ubmap_dir):
    ver_control, software_name, creator = globalVariable()
    folder_det = {}                                         #dict, where the map of the root folder goes.
    max_files = 100
    nu_ub_files = 0
    error = 0
    error_report = {}
    basefolder = os.path.basename(root_directory)             #select the base file
    split = root_directory.split("//")                       #seperate the selected directory w.r.t //
    index_basefolder = split.index(basefolder)                  #index the position of the basefile in the seperated list
    elim_sec = '//'.join(split[:index_basefolder]) + '//'              #join all the element of list except the basefile, which can be eliminated using 
    ubname = str(time.localtime(time.time()).tm_mday) + "." + str(time.localtime(time.time()).tm_mon) + "." + str(time.localtime(time.time()).tm_year)
    rand_folder = ''.join(random.choices(string.ascii_letters + string.digits, k=16))
    rand_name = ''.join(random.choices(string.ascii_letters + string.digits, k=8))
    backup_to = directory_to_backup + "//UB_" + ubname + "//" + basefolder + "//" + rand_folder
    os.makedirs(backup_to)
    for folder, subfolder, files in os.walk(root_directory): #using index of the file. This gives the file to eliminate from the 'folder'.
        if ("$RECYCLE.BIN" or "found.000" or "System Volume Information") not in folder:
            os.chdir(root_directory)
            mod_folder = (folder.replace(elim_sec, '')).replace("\\", "//")
            folder_det[mod_folder] = {}
            try:
                folder_det[mod_folder]["fo_s"], folder_det[mod_folder]["fi_s"], folder_det[mod_folder]["fi_p"] = directoryScan(folder.replace("\\", "//") + "//")
            except:
                error = 1
                error_report[root_directory] = "error in directoryScan function"
                errorSave(working_dir, error_report, folder) 
            try:
                folder_det[mod_folder]["z_n"], folder_det[mod_folder]["k"], nu_ub_files = zipEncryptFiles(folder_det[mod_folder]["fi_s"], folder.replace("\\", "//")  + "//", nu_ub_files, backup_to + "//", working_dir + "//")
            except: 
                error = 1
                error_report[root_directory] = "error in zipEncryptFiles function"
                errorSave(working_dir, error_report, folder.replace("\\", "//"))
            if nu_ub_files > max_files:
                nu_ub_files = 0
                rand_folder = ''.join(random.choices(string.ascii_letters + string.digits, k=16))
                backup_to = directory_to_backup + "//UB_" + ubname + "//" + basefolder + "//" + rand_folder
                os.makedirs(backup_to)
    os.chdir(root_directory)
    try: 
        with open(ubmap_dir + "//" + "UB_" + rand_name + "_" + ubname + ".UBmap", 'w') as logfile:
            try:
                json.dump(folder_det, logfile)   #saving as a json file as json only help to read the dict from the file.
                logfile.close()
            except:
                logfile.close()
                error = 1
                error_report[root_directory] = {}
                error_report[root_directory]["er_dump_.json"] = ubmap_dir + "//" + "UB_" + rand_name + "_" + ubname + ".UBmap"
                errorSave(working_dir, error_report, root_directory)
                print("Gross error Delete the backup files at: " + directory_to_backup + "and Start Again")
                return
    except:
        error = 1
        error_report[root_directory] = {}
        error_report[root_directory]["er_create_.UBmap"] = root_directory + "//" + "UB_" + rand_name + "_" + ubname + ".UBmap"
        errorSave(working_dir, error_report, root_directory)
        print("Gross error Delete the backup files at: " + directory_to_backup + "and Start Again")
        return
    try:
        with open(directory_to_backup + "//UB_" + ubname + "//" + basefolder + "//" + "UB_" + rand_name + "_" + ubname + ".val", 'w') as verif_file:   #verification file... for unzip
            try:
                verif_file.write("V_File Code: " + str(rand_name) + "\n")
                verif_file.write("Created: " + str(ubname) + "\n")
                verif_file.write("Backup Of: " + root_directory + "\n")
                verif_file.write("Backed Up To: " + str(directory_to_backup + "//UB_" + ubname + "//" + basefolder) + "\n")
                verif_file.write("Mod_list: " + "[" + "\"" + "UB_" + rand_name + "_" + ubname + "\"" + "]")
                verif_file.write("\n" + "--"*25 + "\n")
                verif_file.write("This is an automated report generated by " + str(software_name) + " " + str(ver_control) + " " + "Author: " + str(creator))
                verif_file.close()
            except:
                verif_file.close()
                error = 1
                error_report[root_directory] = {}
                error_report[root_directory]["er_write_.val"] = directory_to_backup + "//UB_" + ubname + "//" + basefolder + "//" + "UB_" + rand_name + "_" + ubname + ".val"
                errorSave(working_dir, error_report, directory_to_backup)
                print("Gross error Delete the backup files at: " + directory_to_backup + "and Start Again")
                return
    except:
        error = 1
        error_report[root_directory] = {}
        error_report[root_directory]["er_create_.val"] = directory_to_backup + "//UB_" + ubname + "//" + basefolder + "//" + "UB_" + rand_name + "_" + ubname + ".val"
        errorSave(working_dir, error_report, directory_to_backup)
        print("Gross error Delete the backup files at: " + directory_to_backup + "and Start Again")
        return
    return

In [7]:
def createNewBackup(root_directory, where_to_backup, working_dir, ubmap_directory):
    error = 0
    error_report = {}
    exception_folder = ["$RECYCLE.BIN", "found.000", "System Volume Information"]
    if root_directory.find("\\")!=-1:
        root_directory = root_directory.replace("\\", "//")
        where_to_backup = where_to_backup.replace("\\", "//")
        working_dir = working_dir.replace("\\", "//")
        ubmap_directory = ubmap_directory.replace("\\", "//")
    split = root_directory.split("//")
    if os.path.isdir(root_directory)==False:
        error_report["directory not present"] = root_directory
        return 
    if os.path.isdir(where_to_backup)==False:
        error_report["directory not present"] = where_to_backup
        return 
    if os.path.isdir(ubmap_directory + "//")==False:
        try:
            os.makedirs(ubmap_directory + "//")
        except:
            error_report["er_creat_ubmapdirectory"] = ubmap_directory
            errorSave(working_dir, error_report, ubmap_directory)
            print("Error Creating the UBMap Directory")
            return
    if os.path.isdir(working_dir + "//")==False:
        try:
            os.makedirs(working_dir + "//")
        except:
            error_report["er_creat_workingdir"] = working_dir
            errorSave(working_dir, error_report, working_dir)
            print("Error Creating the working directory")
            return
    if len(split)==2 and split[1]=='':   #condition where the root seems to be inside the dirve, so divide the task and 
        list_folders = os.listdir(root_directory)
        for each_root in list_folders:
            if each_root not in exception_folder:
                if os.path.isdir(root_directory + "//" + each_root)==True:
                    try:
                        newBackup(root_directory + each_root, where_to_backup, working_dir, ubmap_directory)
                    except:
                        error_report[each_root] = "error in newBackup function"
                        errorSave(working_dir, error_report, root_directory + each_root)
    else:
        try:
            newBackup(root_directory, where_to_backup, working_dir, ubmap_directory)
        except:
            error_report[root_directory] = "error in newBackup function"
            errorSave(working_dir, error_report, root_directory)
    print("Success")
    return 

211.70030236244202
